# Notebook Explanation

Evaluate score calibration for the final model.

**Why do we need score calibration?**

Our clients are used to evaluating credit risk based on the traditional FICO range of 300 to 850, with 300 representing the highest risk, 850 the lowest. The outputs of a traditional xgboost model can thus be quite confusing. This work by Moritz Becker serves to make our model outputs more digestible to clients and more comparable to the benchmark scores they typically use, such as FICO and Vantage scores. 

Additionally, a lot of the evaluations in this notebook can be thought of as a second evaluation of your model, not just of the calibration itself. If you have two models that score similarly, this notebook could be used to help choose between them.

**How does it work?**

This will be a bare-bones summary of how score calibration works. For more information, please visit https://zestfinance.atlassian.net/wiki/spaces/DS/pages/1623818294/Zest+Score+Calibration+and+implementation

Score calibration uses two levels of mapping:

1. Map score to risk: This step is specific to each client. Calculate XGBoost scores using the train scores for both national and client data. The scores are divided into 15 equally sized buckets for projects with >= 900k samples, using 1 fewer bucket for each 60k samples below that (with 10 buckets for cases with < 600k samples), and we calculate the mean target rate for each bucket (which we assume equals the mean risk). We then linearly interpolate between these mean target rate points. 

2. Map risk to calibrated score: This step uses one mapping for all clients and models at Zest. This step uses a fixed mapping, which maps risk scores to a score range from 300-850, where 720 represents risk odds of 40:1, and the odds double every 40 points.

The score calibration artifacts for each model were automatically created when you built the model through model-engine. This notebook extracts those artifacts and evaluates the quality of that calibration. The following tests could point to an issue with either the calibration, or the model itself.

# Preparation

## Imports

In [1]:
import copy
import pandas as pd
import numpy as np
import json 
import os
import plotly.express as px

from zaml.common.utils import load_state
from zestio import load_data
from model_engine.validators.zest_score_validation import MappingValidation, ScoreValidation, AUC_analysis
from model_engine.assets.utils import load_asset

/home/jag/.conda/envs/py310/lib/python3.10/site-packages/zaml/common/utils/io.py:17: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
from typing import List

#### Define client and project name

Use project_info.json to load our client and project name. Make sure to update the project_info.json file to match your client and project! 

Also specify the model_id for which you are performing score calibration.

In [4]:
with open('../model_iteration/autoloan/project_info.json', "r") as f:
    project_info = json.load(f)
    
client = project_info['client_name']
bureau = project_info['bureau_production']
product_version = project_info['product_version']
model_iteration_products = project_info['model_iteration_products']

print(client, bureau, model_iteration_products, product_version)

californiacu equifax ['autoloan'] 1


### Connect to ProjectZ (V2)

Quick reminder (collapsible):
- Project Z hierarchy: client > dataset > model_iteration. Higher object need to be created/connected before lower object
- `z_info` serves as a central access point to create, connect to, and manipulate ProjectZ objects
- By this stage the client and dataset should be created by the data requesting team


For more information about ProjectZ V2 object hierarchy and how to interact with `z_info` object, see this [user_guide](https://github.com/Katlean/projectz/blob/c61d8f312f0e687041b0271b7b7c4e3092ee3fd4/user_guide_v2.ipynb)

#### Connect to Client

In [5]:
from projectz import ZInfo
z_info = ZInfo() # Initialize z_info
try:
    z_info.connect_client(name = client) # connect to client level
except ValueError as e:
    print(f"Client doesn't exist. Please double check the input or rerun notebook 0 to create new client\nError message: {e}")

INFO:projectz.logger:Connected client: californiacu
INFO:projectz.logger:Connected client: californiacu
INFO:projectz.logger:Connected to client: californiacu


#### Connect to dataset

In the usual workflow, the arguments to connect to the dataset should be saved to `dataset_info.json` in notebook 0. 
Always confirm the dataset details printed in the cells below before connecting.
If the configuration is outdated or incorrect, rerun Notebook 0 to create a new `dataset_info.json`.

For more explanation for each argument, see the same section in `0_target_and_date_selection.ipynb`

In [6]:
with open("../model_iteration/autoloan/dataset_info.json", "r") as f:
    dataset_info = json.load(f)

dataset_model_type = dataset_info["dataset_model_type"]
dataset_data_source = dataset_info["dataset_data_source"]
dataset_product = dataset_info["dataset_product"]
dataset_version = dataset_info["dataset_version"]
bureau_modeling = dataset_info["bureau_modeling"]

# Sanity check
print(f"""\nLoaded Dataset Configuration:
- model_type: {dataset_model_type}
- Product: {dataset_product}
- Data Source: {dataset_data_source}
- Version: {dataset_version}
- Bureau for Modeling: {bureau_modeling}
""")


Loaded Dataset Configuration:
- model_type: ['underwriting']
- Product: ['autoloan', 'personalloan', 'creditcard', 'homeequity']
- Data Source: ['equifax']
- Version: 1
- Bureau for Modeling: equifax



In [7]:
# Connect to the dataset
try:
    z_info.connect_dataset(
        model_type=dataset_model_type,
        product=dataset_product,
        version=dataset_version
    )
except ValueError:
    print("Dataset not found with the given configuration. Please double check the settings or manually fix in the cell above.")

INFO:projectz.logger:Connected dataset: autoloanCreditcardHomeequityPersonalloanv1
INFO:projectz.logger:Connected dataset: autoloanCreditcardHomeequityPersonalloanv1
INFO:projectz.logger:Connected to dataset: autoloanCreditcardHomeequityPersonalloanv1


#### Connect to model iteration

By this stage you should be sure about the model iteration name. See the section in `1_run_model1.ipynb` for detailed explanation on model iteration.

In [8]:
# ATTENTION: set model iteration name
model_iteration_name: str = 'model1_AppData_LTVAutoloan' # The model for which you are running score calibration
assert model_iteration_name is not None, "You have to set model iteration name"

# Uncomment this part if you want to search for the model iteration with matching product + version
# matching_model_iterations = z_info.context.web_handler.get(f"clients/{z_info.client.id}/datasets/{z_info.dataset.id}/model-iterations")
# for i, model_iter in enumerate(matching_model_iterations):
#     print(f"Available Model No. {i}: {json.dumps(model_iter, indent=4)}")

See notebook `1_run_model.ipynb` for detailed explanation

In [9]:
# ATTENTION: review and inspect model_iteration args; modify if necessary
model_iteration_model_type: List[str] = dataset_model_type # usually same as dataset
model_iteration_data_source: List[str] = dataset_data_source # usually same as dataset
model_iteration_products: List[str] = model_iteration_products # Defined before. Usually a subset of dataset
print(f"""
PLEASE CHECK Model Iteration configuration:
- Model Type: {model_iteration_model_type}
- Data Source: {model_iteration_data_source}
- Product: {model_iteration_products}
- Name: {model_iteration_name}
""")


PLEASE CHECK Model Iteration configuration:
- Model Type: ['underwriting']
- Data Source: ['equifax']
- Product: ['autoloan']
- Name: model1_AppData_LTVAutoloan



In [10]:
# if model iteration exist, connect
# NOTE: you shoudn't need to create a new model iteration here. Please go back to notebook 1 if you want to run a new model
try:
    z_info.connect_model_iteration(
        model_type=model_iteration_model_type,
        product=model_iteration_products,
        name=model_iteration_name
    )
except ValueError:
    print("Dataset not found with the given configuration. Please check the settings above and rerun this cell if they were incorrect.")
    print("If this configuration is new, refer to projectz v2 user guide to create and connect new model iteration")

INFO:projectz.logger:Connected model_iteration: autoloan/model1_AppData_LTVAutoloan
INFO:projectz.logger:Connected model_iteration: autoloan/model1_AppData_LTVAutoloan
INFO:projectz.logger:Connected to model_iteration: autoloan/model1_AppData_LTVAutoloan


## Get model strategy

Specify if the model is a single or ensemble model, to help the validation objects know how to load the data.

In [11]:
modeling_model_artifacts_dir = z_info.model_iteration.paths['modeling_model_artifacts_dir']
try:
    with open(os.path.join(modeling_model_artifacts_dir, "model_strategy.json"), "r") as f:
        model_strategy = json.load(f)
        model_type = model_strategy['model_type']
except Exception as e:
    print(e)
    print('Using manual model_type input. Make sure you adjust this if required.')
    # If there is no model_strategy.json, set it manually 
    model_type = "single"  # For manual input, options are 'single' or 'ensemble'
    
print(f'model_type:{model_type}')

model_type:ensemble


**Load score calibration artifacts**

When running the mapping validation with benchmarking=True, the current validator will not accept any "non-standard" project names (i.e., "autoloanv2", etc.) by default. You can use a non-standard name by passing the standard project into the MappingValidation object, then passing the path to your actual project in the fit method.

The following code uses the fact that typical "non-standard" names are simply a standard name plus a suffix. It will look for the presence of a "standard" project name within your current name, and if it finds one, will use that as the MappingValidation project. If your project name does not contain a "standard" project name, you will need to manually input it into the object.

In [12]:
# map validation
map_projects = ['autoloan', 'personalloan', 'creditcard', 'allproducts']

if len(model_iteration_products) == 1 and model_iteration_products[0] in map_projects: # standard case: single product + product found map_projects
    print(f'Using "{model_iteration_products[0]}" for mapping reference')
    product_calibration = model_iteration_products[0]
else:
    # either multiple products or product not found in map_projects: use allproducts
    print(f"Using allproducts for mapping reference of: {model_iteration_products}")
    product_calibration = "allproducts" # legacy name
# TODO: add z_info back?
mapval=MappingValidation(data_path=modeling_model_artifacts_dir,product=product_calibration,bureau=bureau,model_type=model_type)
mapval.fit(benchmarking=True)

Using "autoloan" for mapping reference


In [13]:
# score validation
scoreval=ScoreValidation(data_path=modeling_model_artifacts_dir)
scoreval.fit(benchmarking=False)

# score comparison with benchmark
scoreval_comp=ScoreValidation(data_path=modeling_model_artifacts_dir)
scoreval_comp.fit(benchmarking=True)

# auc validation
auc_val=AUC_analysis(data_path=modeling_model_artifacts_dir)
auc_val.fit()

In [14]:
if len(model_iteration_products) > 1: # multiproduct
    scoreval_by_prod={}
    auc_val_by_prod={}
    version_suffix = f"v{product_version}" if product_version > 1 else ""
    for single_product in model_iteration_products:
        file = os.path.join(
            '/d/shared/silver_projects/project_power/shared_data/processed/client_data/', # client DB path
            f"{client}_{single_product}{version_suffix}",
            "target.parquet"
        )
        client_data = pd.read_parquet(file)

        keys=client_data.index
        scoreval_by_prod[single_product]=ScoreValidation(data_path=modeling_model_artifacts_dir)
        scoreval_by_prod[single_product].fit(benchmarking=True, key_selection=keys)

        auc_val_by_prod[single_product]=AUC_analysis(data_path=modeling_model_artifacts_dir)
        auc_val_by_prod[single_product].fit(key_selection=keys)

# Looking at source [code](https://github.com/Katlean/model-engine/blob/06c863fd3dfd76f992584a28da0ae3ad01122876/model_engine/validators/zest_score_validation.py#L53C13-L75C1) here for how we actualy create the df for the compariison

```python
if benchmarking==True:

    mapping_objs = load_asset('power/post_sale_path.json')['score_calibration_mapping_objects']

    self.calib_risk_type_bureau_project = pd.read_pickle(os.path.join(mapping_objs, 'calib_risk_type_bureau_project.pkl'))
    
    self.e2e_mapping_type_bureau_project = pd.read_pickle(os.path.join(mapping_objs, 'e2e_mapping_type_bureau_project.pkl'))

    comp_dic={
    'experian':'exp', 
    'transunion':'TU', 
    'equifax':'efx'
    }

    bench_mapping=self.e2e_mapping_type_bureau_project[comp_dic[self.bureau]][self.product][self.model_type]
    bench_mapping=bench_mapping.rename(columns={'score':'src', 'risk':'dstEstimatedRisk', 'calibrated':'dstRecalibratedScore'})
    bench_mapping['Project']='Average across clients'

    dfx=pd.concat([self.e2e_mapping,bench_mapping])
    self.mapping_comparison_display=px.line(dfx,x='src',y='dstEstimatedRisk',color='Project')

    psi_score=PSI()
    psi_score.fit(self.e2e_mapping[['src']])
    self.mapping_comparison_score=psi_score.transform(bench_mapping[['src']])[0]
```

In [16]:
comp_dic={
'experian':'exp', 
'transunion':'TU', 
'equifax':'efx'
}
bench_mapping= mapval.e2e_mapping_type_bureau_project[comp_dic[mapval.bureau]][mapval.product][mapval.model_type]
bench_mapping=bench_mapping.rename(columns={'score':'src', 'risk':'dstEstimatedRisk', 'calibrated':'dstRecalibratedScore'})
bench_mapping['Project']='Average across clients'
dfx=pd.concat([mapval.e2e_mapping,bench_mapping])


In [17]:
comp_dic[mapval.bureau], mapval.product, mapval.model_type

('efx', 'autoloan', 'ensemble')

In [18]:
mapval.e2e_mapping_type_bureau_project.keys()
## has a dictionary of mapping for each product buraue and model type combination

dict_keys(['efx', 'exp', 'TU'])

In [19]:
bench_mapping = mapval.e2e_mapping_type_bureau_project['efx']['autoloan']['ensemble']
bench_mapping=bench_mapping.rename(columns={'score':'src', 'risk':'dstEstimatedRisk', 'calibrated':'dstRecalibratedScore'})
bench_mapping['Project']='Average across clients'

In [20]:
bench_mapping.columns, mapval.e2e_mapping.columns

(Index(['src', 'dstEstimatedRisk', 'dstRecalibratedScore', 'Project'], dtype='object'),
 Index(['src', 'dstRecalibratedScore', 'dstEstimatedRisk', 'Project', 'diff'], dtype='object'))

In [21]:
from zaml.analyze.data_analysis.distribution_drift import PSI
psi_score=PSI()
psi_score.fit(mapval.e2e_mapping[['src']])

In [160]:
ls /d/shared/silver_projects_v2/californiacu/autoloanCreditcardHomeequityPersonalloanv1/modeling/model_artifacts/autoloan/model1_AppData_LTVAutoloan

app/                          performance.parquet
artifact_manifest.json        pipeline_equifax.obj
asset.json                    pipeline_experian.obj
calibration_object.obj        pipeline.obj
client_predictor_config.json  pipeline_transunion.obj
data_description.json         power_model_config.json
feature_definition.parquet    run_time.json
feature_importance.parquet    score_recalibration_mapping.json
fe_data/                      splitter.obj
inject_model_config.json      submodel_feature_importance/
input_configuration.json      submodel_performance.parquet
keep_features.json            submodel_scores/
key_factors_mapping.json      target/
model.obj                     test_data_summary.json
model_strategy.json           unfold_data_description.json
overall_scores/               value_based_key_factor_mapping.json
overall_zest_scores/          versions.json


In [23]:
import pickle

base_path = '/d/shared/silver_projects_v2/californiacu/autoloanCreditcardHomeequityPersonalloanv1/modeling/model_artifacts/autoloan/model1_AppData_LTVAutoloan/'
name = 'calibration_object.obj'

def load_pickle(base_path, name):
    path = f'{base_path}{name}'
    with open(path, 'rb') as f:
        meta = pickle.load(f)
        object = pickle.load(f)
    return object
calibration_object = load_pickle(base_path= base_path, name = name)


Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml




In [ ]:
calibration_object

In [172]:
calibration_object.__dict__.keys()

dict_keys(['n_tiers', 'dst_risk_score_mapping', 'epsilon', 'src_min', 'src_max', 'dst_min', 'dst_max', 'dst_placeholders', 'src_placeholders', 'range_error', '_dst_to_intermediate_interf', '_intermediate_to_src_interf', '_src_to_intermediate_interf', '_intermediate_to_dst_interf', 'src_risk_score_mapping', '_src_quantile_mapping'])

### calibration object has mapping here which for bucekts of 15

In [173]:
calibration_object._src_quantile_mapping

,score,risk
0,0.000000,0.000000
1,0.184825,0.000732
2,0.187202,0.001965
3,0.190311,0.003239
4,0.194713,0.005828
5,0.200892,0.009053
6,0.209297,0.013471
7,0.220272,0.021603
8,0.234151,0.030412
9,0.251303,0.046404


In [171]:
for key in calibration_object.__dict__.keys():
    print(f'key: {key}')
    output = calibration_object.__dict__[key]
    print(output)
    print(f'\n')

key: n_tiers
15


key: dst_risk_score_mapping
     score      risk
550    300  1.000000
549    301  0.649195
548    302  0.648305
547    303  0.647415
546    304  0.646526
..     ...       ...
4      846  0.000104
3      847  0.000069
2      848  0.000040
1      849  0.000016
0      850  0.000000

[551 rows x 2 columns]


key: epsilon
0.0001


key: src_min
0


key: src_max
1


key: dst_min
300


key: dst_max
850


key: dst_placeholders
[]


key: src_placeholders
[]


key: range_error
True


key: _dst_to_intermediate_interf


key: _intermediate_to_src_interf


key: _src_to_intermediate_interf


key: _intermediate_to_dst_interf


key: src_risk_score_mapping
None


key: _src_quantile_mapping
       score      risk
0   0.000000  0.000000
1   0.184825  0.000732
2   0.187202  0.001965
3   0.190311  0.003239
4   0.194713  0.005828
5   0.200892  0.009053
6   0.209297  0.013471
7   0.220272  0.021603
8   0.234151  0.030412
9   0.251303  0.046404
10  0.272442  0.061813
11  0.298949  0.088011
12 

# Basically, when we build our calibration object, the first thing we do is we split our scores in 14 buckets of equal size




## This code is found [here](https://github.com/Katlean/zaml/blob/1270c79eedb3129a3e042577efe0e373a41f1d6e/zaml/model/modeling/calibration.py#L610C4-L634C24) in the _quantiles_risk_estimator function in the FixedRiskScoreRecalibration Class in zaml which ZestScoreCalibration inherits from


```python
def _quantiles_risk_estimator(self, score, target, sc_min, sc_max):
        d_scores = pd.DataFrame({"score": score, "target": target})
        d_scores["tier"] = pd.qcut(d_scores["score"], q=self.n_tiers - 1, precision=10)
        tiers = d_scores.groupby(["tier"]).mean().reset_index()
        tiers.sort_values(by=["tier"]).reset_index(drop=True)
        risk_est = tiers.drop("tier", inplace=False, axis=1)
        risk_est.rename(columns={"target": "risk"}, inplace=True)

        risk_est.sort_values(by="score", inplace=True)
        decreasing = (
            risk_est.risk.iloc[0] > risk_est.risk.iloc[-1]
        )  # if decreasing function target(score)
        # add minimum
        sc_min = min(score) if sc_min is None else sc_min
        sc_min_row = pd.DataFrame(
            [[sc_min, int(decreasing)]], columns=["score", "risk"]
        )
        risk_est = pd.concat([sc_min_row, risk_est], ignore_index=True)
        sc_max = max(score) if sc_max is None else sc_max
        sc_max_row = pd.DataFrame(
            [[sc_max, int(not decreasing)]], columns=["score", "risk"]
        )
        risk_est = pd.concat([risk_est, sc_max_row], ignore_index=True)
        risk_est["risk"] = self._ensure_monotonicity(risk_est["risk"], decreasing)
        return risk_est
```

When we call [fit](https://github.com/Katlean/model-engine/blob/06c863fd3dfd76f992584a28da0ae3ad01122876/model_engine/model_builder/artifacts/zest_score.py#L33C2-L46C81) for our zest score calibration object

```python
  def fit(self, train_scores, train_target):
        # train target or scores might be a list of dataframe/series or a single dataframe/series; we need to vertically concat them if they are in a list
        train_target_union = pd.concat(train_target, axis=0) if isinstance(train_target, list) else train_target
        train_scores_union = pd.concat(train_scores, axis=0) if isinstance(train_scores, list) else train_scores

        train_target_union = train_target_union[train_target_union.notna()]
        
        mask = train_scores_union.index.intersection(train_target_union.index)
        train_scores_union = train_scores_union.filter(items=mask, axis=0)
        train_target_union = train_target_union.filter(items=mask, axis=0)

        train_scores_union = train_scores_union.loc[train_target_union.index]
        
        super().fit(score_src=train_scores_union, target_src=train_target_union)
```

We call fit inherited which is FixedRiskScoreRecalibration which does not have a fit but inherits from ScoreRecalibrationBase

which calls the [_fit](https://github.com/Katlean/zaml/blob/1270c79eedb3129a3e042577efe0e373a41f1d6e/zaml/model/modeling/calibration.py#L354C7-L359C10) which is in FixedRiskScoreRecalibration [here](https://github.com/Katlean/zaml/blob/1270c79eedb3129a3e042577efe0e373a41f1d6e/zaml/model/modeling/calibration.py#L669C5-L725C20)

```python
 self._dst_quantile_mapping = self._quantiles_risk_estimator(
                score_dst, target_dst, self.dst_min, self.dst_max
            )
```

We can see this creates the mapping such it has 0 1 at the bounds and in between it has all the 14 and for each of those it has the average score and the average target rate

In [24]:
calibration_object._src_quantile_mapping

,score,risk
0,0.000000,0.000000
1,0.184825,0.000732
2,0.187202,0.001965
3,0.190311,0.003239
4,0.194713,0.005828
5,0.200892,0.009053
6,0.209297,0.013471
7,0.220272,0.021603
8,0.234151,0.030412
9,0.251303,0.046404


# We then linearly intrpoalte between this to get everyone's estimate risk 

# TO get the zest score we then take that risk and map it into general_mapping.json to get the final risk score

## this is fixed!!!

That maps odds of roughly 45:1 which under the odds-doubling convention (40:1 at 720, doubling every 40 points) lands around a Zest score of ~725

we can see it loaded [here](https://github.com/Katlean/model-engine/blob/06c863fd3dfd76f992584a28da0ae3ad01122876/model_engine/model_builder/artifacts/zest_score.py#L136C12-L136C72)

```python
 dst_min, dst_max = int(min(map.keys())), int(max(map.keys()))

        calib_score_range = np.arange(dst_max, dst_min-1, -1)
        e2e_mapping=calibration_object.inverse_transform(calib_score_range)
        e2e_mapping=e2e_mapping.rename(columns={'score':'src','risk':'dstEstimatedRisk','zest_score':'dstRecalibratedScore'})
        e2e_mapping.sort_values('src',inplace=True)

        score_recalibration_mapping=e2e_mapping[['src','dstRecalibratedScore','dstEstimatedRisk']].to_dict(orient='list')
```


## Thus that final e2e_mapping object we use in the PSI is abscialyl taking for every zest score what would have been the original model prediction to have generated that.

## so when we do the PSI we are basically comparing hwo different the mapping is from original score to final zest score between our model and an average of models of the same type (national ensamble buraeu product)

In [30]:
cd playground/


This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.




/home/jag/client-project-californiacu/autoloanCreditcardHomeequityPersonalloanv1/playground


In [27]:
ls ../..

0_target_and_date_selection.ipynb
claude_testing.ipynb
client-project-bestegg/
client-project-californiacu/
client-project-common/
client-project-members1stfcu/
client-project-patelcocu/
client-project-penfed/
client-project-westmark/
code/
Create_Model_Iteration_Onboarding_Guide.pdf
delete_model_iteration_guide.md
demoanalysis/
demoanalysis_backup_20260122_1751/
feature-engine-parts/
mega_model_config_analysis.pdf
model-engine/
model_engine_version_compatibility.md
model_iteration_management_examples.py
national_model_trace_log.md
newest_model_engine_env_backup.yml
newest_model_engine_kernelspec_backup/
Onboarding/
presentable-engine/
projects/
projectz/
QUICK_ANSWER_Delete_Model_Iteration.txt
QUICK_ANSWER_me_version_compatibility.txt
QUICK_ANSWER_skip_argument_validation.txt
README.md
report-engine/
skip_argument_validation_explained.md
test/
Testing_inquiry_and_trade_dables.ipynb
Untitled10.ipynb
Untitled1.ipynb
untitled1.txt
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5